In [2]:
import os

os.chdir("/home/tim/Development/OCPPM/")
import pickle
import numpy as np
import random
import pandas as pd
import torch
import torch_geometric.nn as pygnn
import torch_geometric.transforms as T
import torch_geometric.loader as L
import torch.nn.functional as F
import torch.optim as O

# Custom imports
from loan_application_experiment.models.geometric_models import (
    AGNN_EFG,
    HigherOrderGNN_EFG,
    GraphModel,
    SimpleGNN_EFG,
)

import utilities.evaluation_utils as evaluation_utils
import utilities.hetero_data_utils as hetero_data_utils
import utilities.hetero_evaluation_utils as hetero_evaluation_utils
import utilities.hetero_training_utils as hetero_training_utils
import utilities.torch_utils as torch_utils

# Print system info
torch_utils.print_system_info()
torch_utils.print_torch_info()


def quick_check(t: torch.Tensor) -> pd.DataFrame:
    return pd.DataFrame(t).head()

CPU: Intel(R) Core(TM) i5-7500 CPU @ 3.40GHz (4x)
Total CPU memory: 46.93GB
Available CPU memory: 38.64GB
GPU: NVIDIA GeForce GTX 960
Total GPU memory: 4096.0MB
Available GPU memory: 4029.0MB
Platform: Linux-5.19.0-46-generic-x86_64-with-glibc2.35
Torch version: 1.13.1+cu117
Cuda available: True
Torch geometric version: 2.3.1


In [12]:
objects_data_dict_file = "data/BPI17/feature_encodings/OFG/ofg/raw/BPI17_OFG.pkl"
with open(objects_data_dict_file, "rb") as fp:
    ofg = pickle.load(fp)
data = ofg
data = T.ToUndirected()(data)
data = T.AddSelfLoops()(data)
data = T.NormalizeFeatures()(data)
data

HeteroData(
  application={
    y=[31509],
    x=[31509, 3]
  },
  offer={
    y=[42995],
    x=[42995, 7]
  },
  (application, interacts, application)={ edge_index=[2, 31509] },
  (application, interacts, offer)={ edge_index=[2, 42995] },
  (offer, interacts, offer)={ edge_index=[2, 55049] },
  (offer, rev_interacts, application)={ edge_index=[2, 42995] }
)

In [14]:
class HigherOrderGNN_HOEG(GraphModel):
    def __init__(self, hidden_channels: int = 64, out_channels: int = 1):
        super().__init__()
        self.conv1 = pygnn.GraphConv(-1, hidden_channels)
        self.conv2 = pygnn.GraphConv(-1, hidden_channels)
        self.act1 = torch.nn.PReLU()
        self.act2 = torch.nn.PReLU()
        # self.pool1 = pygnn.global_add_pool
        self.out = pygnn.Linear(-1, out_channels)

    def forward(self, x, edge_index, batch):
        x = self.conv1(x, edge_index)
        x = self.act1(x)
        x = self.conv2(x, edge_index)
        x = self.act2(x)
        # x = self.pool1(x, batch)
        x = self.out(x)
        return x


model = HigherOrderGNN_HOEG(32, 1)
model = pygnn.to_hetero(model, data.metadata())
# model.double()

# with torch.no_grad():  # Initialize lazy modules.
#      out = model(het_data.x_dict, het_data.edge_index_dict)

/home/tim/Development/OCPPM/.env/lib/python3.9/site-packages/torch_geometric/nn/to_hetero_transformer.py:379: UserWarning: 'act1' will be duplicated, but its parameters cannot be reset. To suppress this warning, add a 'reset_parameters()' method to 'act1'
  warnings.warn(
/home/tim/Development/OCPPM/.env/lib/python3.9/site-packages/torch_geometric/nn/to_hetero_transformer.py:379: UserWarning: 'act2' will be duplicated, but its parameters cannot be reset. To suppress this warning, add a 'reset_parameters()' method to 'act2'
  warnings.warn(


In [9]:
# from torch_geometric.sampler import HeteroSamplerOutput, HGTSampler
# from torch_geometric.loader import HGTLoader

# het_data = T.RandomNodeSplit()(het_data)
# loader = HGTLoader(het_data,
#                    num_samples={key: [10_000] for key in het_data.node_types},
#                    input_nodes=('offer', het_data['offer'].train_mask))

In [10]:
het_data = T.RandomNodeSplit()(het_data)

optimizer = O.Adam(
    model.parameters(),
    lr=0.001,
    betas=(0.9, 0.999),
    eps=1e-08,
    weight_decay=0,
    amsgrad=False,
)
loss = train(model, het_data, optimizer, F.l1_loss)
loss

/tmp/ipykernel_30541/3943607108.py:6: UserWarning: Using a target size (torch.Size([41495])) that is different to the input size (torch.Size([41495, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  loss = loss_fn(out["offer"][mask], data["offer"].y[mask])


: 

: 

In [ ]:
loss
# 1.3763950517127197

10588784.253657103

In [ ]:
net.

AssertionError: 